# Run and process the prior monte carlo


In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams['font.size']=12
import flopy
import pyemu
%matplotlib inline

## SUPER IMPORTANT: SET HOW MANY PARALLEL WORKERS TO USE

In [ ]:
num_workers = 20

### set the `t_d` or "template directory" variable to point at the template folder and read in the PEST control file

In [ ]:
t_d = "template"
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))

In [ ]:
pst.npar_adj

Load the previously generated parameter ensemble and inspect (again!)...

In [ ]:
pe = pyemu.ParameterEnsemble.from_binary(pst=pst,filename=os.path.join(t_d,"prior.jcb"))
#pe.loc[:,should_fix] = 1.0
pe.to_csv(os.path.join(t_d,"sweep_in.csv"))
pe.shape

In [ ]:
pe.loc[:,"hk031"]

In [ ]:
pe.loc[:,"hk031"].hist(bins=50)

look! hk is log-normal-ish

Lets run the first realization through the pest interface for a test:

In [ ]:
# replace the par vals with the first row in the par ensemble
pst.parameter_data.loc[pe.columns,"parval1"] = pe.iloc[0,:]
pst.control_data.noptmax = 0
pst.write(os.path.join(t_d,"test.pst"))
pyemu.os_utils.run("pestpp-ies test.pst",cwd=t_d)
res = pyemu.pst_utils.read_resfile(os.path.join(t_d,"test.base.rei"))
res.loc[pst.nnz_obs_names,:]

### run the prior ensemble in parallel locally
This takes advantage of the program `pestpp-swp` which runs a parameter sweep through a set of parameters. By default, `pestpp-swp` reads in the ensemble from a file called `sweep_in.csv` which in this case we made just above.

In [ ]:
m_d = "master_prior_sweep"
pyemu.os_utils.start_workers(t_d,"pestpp-swp","freyberg.pst",num_workers=num_workers,worker_root=".",master_dir=m_d)

### Load the output ensemble and plot a few things



In [ ]:
obs_df = pd.read_csv(os.path.join(m_d,"sweep_out.csv"),index_col=0)
print('number of realization in the ensemble before dropping: ' + str(obs_df.shape[0]))

### drop any failed runs 

In [ ]:
obs_df = obs_df.loc[obs_df.failed_flag==0,:]
print('number of realization in the ensemble **after** dropping: ' + str(obs_df.shape[0]))

In [ ]:
obs_df.iloc[0,:]

### confirm which quantities were identified as forecasts

In [ ]:
fnames = pst.pestpp_options["forecasts"].split(',')
fnames

We now see that the most extreme scenario yields a large decrease in flow from the aquifer to the headwaters (the most negative value).  

### Many modeling analyses could stop right here to avoid the ill-effects of history matching...

In [ ]:
obs = pst.observation_data
plt.figure()
for forecast in fnames:
    ax = plt.subplot(111)
    obs_df.loc[:,forecast].hist(ax=ax,color="0.5",alpha=0.5)
    ax.plot([obs.loc[forecast,"obsval"],obs.loc[forecast,"obsval"]],ax.get_ylim(),"r")
    ax.set_title(forecast)
    plt.show()

observations:

In [ ]:
for oname in pst.nnz_obs_names:
    ax = plt.subplot(111)
    obs_df.loc[:,oname].hist(ax=ax,color="0.5",alpha=0.5)
    ax.plot([obs.loc[oname,"obsval"],obs.loc[oname,"obsval"]],ax.get_ylim(),"r")
    ax.set_title(oname)
    plt.show()

Write the "true" obsvals to the obs loc info

In [ ]:
b_d = os.path.join("..","..","data","freyberg_nwt")

In [ ]:
nz_obs = obs.loc[pst.nnz_obs_names,:]
nz_obs.to_csv(os.path.join(b_d,"obs_data.csv"))